**Objective of the notebook:**

In this notebook, let us explore the given dataset and make some inferences along the way. Also finally we will build a baseline light gbm model to get started. 

**Objective of the competition:**

In this competition, we a’re challenged to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. 

In [2]:
import os
#os.getcwd()
os.chdir("/Users/JidaThada/Desktop/2018 Fall/Machine Learning/Project/google_analytics")

In [3]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_selection import f_classif
#import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# Run RNN

In [4]:
processed_train_df = pd.read_csv("processed_train_df.csv", dtype={'fullVisitorId': 'str'})
processed_test_df = pd.read_csv("processed_test_df.csv", dtype={'fullVisitorId': 'str'})

In [5]:
processed_train_df = processed_train_df.drop('Unnamed: 0', axis=1)
processed_test_df = processed_test_df.drop('Unnamed: 0', axis=1)

In [6]:
train_df = pd.read_csv("train_flatten.csv", dtype={'fullVisitorId': 'str'})
test_df = pd.read_csv("test_flatten.csv", dtype={'fullVisitorId': 'str'})

/Users/JidaThada/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.



In [7]:
processed_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208


In [15]:
mask = processed_train_df.fullVisitorId.groupby('fullVisitorId').transform('value_counts') > 1

KeyError: 'fullVisitorId'

In [12]:
index_dev = train_df.index[train_df['date'] <= 20170531]
len(index_dev)
index_val = train_df.index[train_df['date'] > 20170531]
len(index_val)

137946

In [13]:
X_train = processed_train_df.iloc[:,2:]
X_train_dev = X_train.iloc[index_dev,:]
X_train_val = X_train.iloc[index_val,:]
Y_train = processed_train_df['totals.transactionRevenue']
Y_train_dev = Y_train.iloc[index_dev]
Y_train_val = Y_train.iloc[index_val]

X_test = processed_test_df.iloc[:,1:]

In [14]:
log_Y_train = np.log1p(Y_train)
log_Y_train_dev = log_Y_train.iloc[index_dev]
log_Y_train_val = log_Y_train.iloc[index_val]

In [15]:
max(log_Y_train)

23.86437469605166

In [16]:
# normalize 
from sklearn import preprocessing

#X_train_scaled = scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [17]:
X_train_scaled = pd.DataFrame(X_train_scaled)
X_train_scaled.columns = X_train.columns
X_train_scaled_dev = X_train_scaled.iloc[index_dev,:]
X_train_scaled_val = X_train_scaled.iloc[index_val,:]

### 5-fold

In [23]:
processed_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208


In [30]:
unique_visitorId = list(set(processed_train_df['fullVisitorId']))
no_cust = len(unique_visitorId)


In [45]:
id_cv0 = unique_visitorId[0:no_cust//5]
id_cv1 = unique_visitorId[no_cust//5:2*(no_cust//5)]
id_cv2 = unique_visitorId[2*(no_cust//5):3*(no_cust//5)]
id_cv3 = unique_visitorId[3*(no_cust//5):4*(no_cust//5)]
id_cv4 = unique_visitorId[4*(no_cust//5):no_cust]

fold = 5
id_cv = []
for i in range(fold):
    if i<fold-1:
        cur_cv = unique_visitorId[i*(no_cust//5):(i+1)*(no_cust//5)]
    else:
        cur_cv = unique_visitorId[i*(no_cust//5):no_cust]
    id_cv.append(cur_cv)
for e in id_cv:
    print(len(e))
    

142833
142833
142833
142833
142835


In [32]:
len(id_cv0)+len(id_cv1)+len(id_cv2)+len(id_cv3)+len(id_cv4)

714167

In [39]:
for i in range(fold):
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]


714167


### Linear regression

In [52]:
train.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208
5,2938943183656635653,0.0,1,1.0,1,8,9,0,1,4,20,955,100,122,12548,482,19,4,11,5,3196,208
6,1905672039242460897,0.0,1,1.0,1,8,11,0,1,4,20,955,154,122,38725,482,18,4,11,5,3196,208
7,537222803633850821,0.0,1,1.0,1,8,10,0,1,4,20,125,12,0,4340,339,1,4,11,5,3196,208


In [76]:
linear_reg_Y_train_dev_pred[0:10]

array([-0.20094697,  0.20795687, -0.18524478, -0.14895977, -0.18924344,
       -0.1392365 , -0.11650071, -0.15730181, -0.07781301, -0.12514533])

In [88]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    linear_reg = LinearRegression().fit(x_tr, log_y_tr)
    log_y_tr_pred = linear_reg.predict(x_tr)
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = linear_reg.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))





fold: 0
train_mse 3.3704845503164003
train_rmse 1.8358879460131547
val_mse 3.3474725970656345
val_rmse 1.829609957631854


fold: 1
train_mse 3.3750687113447295
train_rmse 1.8371360078515497
val_mse 3.3505344463023357
val_rmse 1.8304465155536054


fold: 2
train_mse 3.36008079186025
train_rmse 1.8330523156364769
val_mse 3.401107484509341
val_rmse 1.8442091759096475


fold: 3
train_mse 3.364309668380293
train_rmse 1.8342054596964574
val_mse 3.3780034678279294
val_rmse 1.8379345657089996


fold: 4
train_mse 3.367373678720298
train_rmse 1.8350405114657
val_mse 3.3636193020598655
val_rmse 1.8340172578413392


Average:
train_mse_5fold 3.367463480124394
train_rmse_5fold 1.8350644481326674
val_mse_5fold 3.368147459553021
val_rmse_5fold 1.8352434945290892


In [87]:
def getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred):
    revenue = np.exp(log_y_tr_pred) - 1
    id_list = list(train['fullVisitorId'])
    
    d = {'fullVisitorId':id_list, 'PredictedRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedRevenue"].sum().reset_index())
    
    
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedRevenue'])
    y_tr_pred = list(submit['PredictedLogRevenue'])
    
    y_train_sumrev = pd.DataFrame(train.groupby('fullVisitorId')["totals.transactionRevenue"].sum().reset_index())
    y_train_sumrev['totals.transactionRevenue'] = np.log1p(y_train_sumrev['totals.transactionRevenue'])
    y_tr = list(y_train_sumrev['totals.transactionRevenue'])
    
    mse_tr = mean_squared_error(y_tr, y_tr_pred)
    print('train_mse', mse_tr)
    print('train_rmse', np.sqrt(mse_tr))

    
    revenue = np.exp(log_y_val_pred) - 1
    id_list = list(val['fullVisitorId'])
    
    d = {'fullVisitorId':id_list, 'PredictedRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedRevenue"].sum().reset_index())
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedRevenue'])
    y_val_pred = list(submit['PredictedLogRevenue'])
    
    y_val_sumrev = pd.DataFrame(val.groupby('fullVisitorId')["totals.transactionRevenue"].sum().reset_index())
    y_val_sumrev['totals.transactionRevenue'] = np.log1p(y_val_sumrev['totals.transactionRevenue'])
    y_val = list(y_val_sumrev['totals.transactionRevenue'])
    
    mse_val = mean_squared_error(y_val, y_val_pred)
    
    print('val_mse', mse_val)
    print('val_rmse', np.sqrt(mse_val))
    return mse_tr, mse_val



In [19]:
# Test set
# processed_test_df

linear_reg = LinearRegression().fit(x_tr, log_y_tr)

x_te = processed_test_df.iloc[:,1:]
te_id = processed_test_df.iloc[:,0]

y_te_pred = poly_reg.predict(x_te)
y_te_pred =[0 if i < 0 else i for i in y_te_pred]
revenue = np.exp(y_te_pred) - 1

# PredictedLogRevenue
d = {'fullVisitorId':te_id, 'PredictedLogRevenue':revenue}
submit = pd.DataFrame(data=d)
col = ['fullVisitorId', 'PredictedLogRevenue']
submit = submit[col]
submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedLogRevenue"].sum().reset_index())
submit['PredictedLogRevenue'] = np.log1p(submit['PredictedLogRevenue'])
print(submit.shape)
submit.head()
submit.to_csv('submit1.csv', index=False)

/Users/JidaThada/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:14: RuntimeWarning:

invalid value encountered in log1p



(617242, 2)


In [20]:
submit.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.503747
1,0000049363351866189,0.253831
2,0000053049821714864,-0.229559
3,0000059488412965267,0.565368
4,0000085840370633780,-0.344739


### Polynomial Regression

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [89]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    model = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
    poly_reg = model.fit(x_tr, log_y_tr)
    log_y_tr_pred = poly_reg.predict(x_tr)
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = poly_reg.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))






fold: 0
train_mse 2.955223531070177
train_rmse 1.7190763598718286
val_mse 2.929755986228277
val_rmse 1.7116529981945163


fold: 1
train_mse 2.9506842924557968
train_rmse 1.7177555974165233
val_mse 2.953717193699767
val_rmse 1.7186381799843058


fold: 2
train_mse 2.954605705688796
train_rmse 1.7188966535800796
val_mse 2.9660912159707586
val_rmse 1.7222343673178626


fold: 3
train_mse 2.950591708901739
train_rmse 1.7177286482159337
val_mse 2.963602552989965
val_rmse 1.7215117057371307


fold: 4
train_mse 2.9365843108844847
train_rmse 1.7136464953089026
val_mse 2.9778602660324536
val_rmse 1.7256477815685487


Average:
train_mse_5fold 2.9495379098001986
train_rmse_5fold 1.7174207508786534
val_mse_5fold 2.9582054429842444
val_rmse_5fold 1.7199370065604729


In [91]:
processed_test_df.head()

,fullVisitorId,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.newVisits,cat.totals.bounces,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,6167871330617112363,4.0,4.0,2.0,9,10,0,1,4,7,1,176,0,24691,0,16,4,11,5,3196,208
1,0643697640977915618,5.0,5.0,1.0,9,17,1,1,4,20,948,185,0,30734,23,19,4,11,5,3196,208
2,6059383810968229466,7.0,7.0,1.0,9,8,1,1,4,7,955,69,122,32535,482,22,4,11,5,3196,208
3,2376720078563423631,8.0,4.0,1.0,9,22,1,1,4,23,540,218,100,0,73,12,4,11,5,3196,208
4,2314544520795440038,9.0,4.0,1.0,9,5,1,1,4,7,751,218,100,0,73,12,4,11,5,3196,208


In [93]:
# Test set
# processed_test_df

x_tr = processed_train_df.iloc[:,2:]
y_tr = processed_train_df.iloc[:,1]
log_y_tr = np.log1p(y_tr)

model = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
poly_reg = model.fit(x_tr, log_y_tr)

x_te = processed_test_df.iloc[:,1:]
te_id = processed_test_df.iloc[:,0]

y_te_pred = poly_reg.predict(x_te)
y_te_pred =[0 if i < 0 else i for i in y_te_pred]
revenue = np.exp(y_te_pred) - 1

# PredictedLogRevenue
d = {'fullVisitorId':te_id, 'PredictedLogRevenue':revenue}
submit = pd.DataFrame(data=d)
col = ['fullVisitorId', 'PredictedLogRevenue']
submit = submit[col]
submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedLogRevenue"].sum().reset_index())
submit['PredictedLogRevenue'] = np.log1p(submit['PredictedLogRevenue'])
print(submit.shape)
submit.head()
submit.to_csv('submit2.csv', index=False)

(617242, 2)


### Classification + RNN

In [94]:
processed_train_df.head()

,fullVisitorId,totals.transactionRevenue,num.totals.hits,num.totals.pageviews,num.visitNumber,cat.date,cat.visitStartTime,cat.totals.bounces,cat.totals.newVisits,cat.channelGrouping,cat.device.operatingSystem,cat.geoNetwork.city,cat.geoNetwork.country,cat.geoNetwork.metro,cat.geoNetwork.networkDomain,cat.geoNetwork.region,cat.geoNetwork.subContinent,cat.trafficSource.campaign,cat.trafficSource.keyword,cat.trafficSource.medium,cat.trafficSource.referralPath,cat.trafficSource.source
0,1131660440785968503,0.0,1,1.0,1,8,15,0,1,4,20,378,210,0,37454,193,21,4,11,5,3196,208
1,377306020877927890,0.0,1,1.0,1,8,5,0,1,4,7,955,12,122,10098,482,1,4,11,5,3196,208
2,3895546263509774583,0.0,1,1.0,1,8,1,0,1,4,20,475,185,0,38725,99,19,4,11,5,3196,208
3,4763447161404445595,0.0,1,1.0,1,8,5,0,1,4,6,955,94,122,38725,482,16,4,1607,5,3196,208
4,27294437909732085,0.0,1,1.0,2,8,13,0,0,4,1,955,217,122,38725,482,13,4,11,5,3196,208
